In [16]:
import pandas as pd

In [17]:
import numpy as np
from dateutil import parser
from scipy.fftpack import fft
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
from scipy import signal

In [18]:
path = '/Users/Susie/Documents/PT17.0105/PT17.0105_01.csv'
epoch = '2s' #e.g: 10s,20s
highcut = 4
lowcut = 0.2
fs = 62.5
order = 5

In [19]:
def loadData():
    sensorData = pd.read_csv(path,low_memory=True)
    sensorData['time1'] = pd.to_datetime(sensorData['time'], unit='ms')
    sensorData['time1'] = sensorData['time1'].apply(lambda x: x.strftime('%H:%M:%S'))
    return sensorData

In [91]:
sensor_data = loadData()
sensor_data.head(5)
sensor_data = sensor_data[np.isfinite(sensor_data['accelerometerX'])]
sensor_data = sensor_data[np.isfinite(sensor_data['accelerometerY'])]
sensor_data = sensor_data[np.isfinite(sensor_data['accelerometerZ'])]
sensor_data = sensor_data[np.isfinite(sensor_data['time'])]

## high pass filter, low pass filter and band pass filter

In [93]:
# High Pass filter configuration
def highpass(high, fs, order):
    nyq = 0.5 * fs
    cutoff = high / nyq
    b, a = butter(order, cutoff, btype='high', analog=False)
    return b, a

# Function to perform high pass filtering
def highpass_filter(data, highcut, fs, order):
    b, a = highpass(highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [94]:
# Low Pass filter configuration

def lowpass(low, fs, order):
    nyq = 0.5 * fs
    cutoff = low / nyq
    b, a = butter(order, cutoff, btype='low', analog=False)
    return b, a
# Function to perform low pass filtering

def lowpass_filter(data, lowcut, fs, order):
    b, a = lowpass(lowcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [95]:
def BandPass(lowcut,highcut,fs,order):
    nyq = fs*0.5
    low = lowcut/nyq
    high = highcut/nyq
    b,a = butter(order,[low,high],btype='bandpass')
    return b,a

def BandPassFilter(data,lowcut,highcut,fs,order):
    b,a = BandPass(lowcut,highcut,fs,order)
    y = lfilter(b,a,data)
    return y

In [101]:
def groupData(sensorData):
    sensorData = sensorData.set_index(sensorData['time1'].map(parser.parse))
    grouped = sensorData.groupby(pd.TimeGrouper(epoch))
    return grouped

In [103]:
def getMSP(inputdata):
    ax = BandPassFilter(data = inputdata, lowcut= lowcut,highcut = highcut, fs = fs, order = order)
    fax,AX = signal.periodogram(ax,fs=fs,scaling='density',return_onesided=True)
    
    periodogram_ax = pd.DataFrame()
    periodogram_ax['freq'] = fax
    periodogram_ax['spec'] = AX
    periodogram_ax = periodogram_ax[(periodogram_ax['freq']>=0.2)&(periodogram_ax['freq']<=4)]
    
    wholesp = periodogram_ax['spec'].sum()
    n = periodogram_ax.shape[0]
    return wholesp,n

In [102]:
grouped = groupData(sensor_data)

In [118]:
SumspX = grouped['accelerometerX'].apply(lambda x: getMSP(x))
SumspY = grouped['accelerometerY'].apply(lambda x: getMSP(x))
SumspZ = grouped['accelerometerZ'].apply(lambda x: getMSP(x))

In [119]:
SSPX = SumspX.tolist()
SSPY = SumspY.tolist()
SSPZ = SumspZ.tolist()
frame_x = pd.DataFrame(SSPX, columns=['sumX', 'number'])
frame_y = pd.DataFrame(SSPY, columns=['sumY', 'number'])
frame_z = pd.DataFrame(SSPZ, columns=['sumZ', 'number'])

In [120]:
frame = pd.DataFrame()
frame['sum'] = frame_x['sumX']+frame_y['sumY']+frame_z['sumZ']
frame['number'] = frame_x['number']+frame_y['number']+frame_z['number']
frame['MSP'] = frame['sum'] / frame['number']
frame.head(5)

,sum,number,MSP
0,0.156728,21,0.007463
1,0.158866,24,0.006619
2,0.158741,24,0.006614
3,0.158816,24,0.006617
4,0.158989,24,0.006625
